In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import xlwt
import pandas as pd
import numpy as np
from selenium.webdriver import ActionChains


In [109]:
domain = 'https://www.tripadvisor.com'
target_url = 'https://www.tripadvisor.com/Attractions'
opt = webdriver.ChromeOptions()
# opt.set_headless()
opt.add_experimental_option('excludeSwitches', ['enable-automation']) # Turn on developer mode
driver = webdriver.Chrome('/Users/haorongzhu/Downloads/chromedriver', options = opt)


destination = 'London, United Kindom'
# dep_date = '12/12/2020'
# ret_date = '16/12/2020'

# step 1: location picker
try:
    wait = WebDriverWait(driver, 40)
    driver.get(target_url)

    location_picker = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="taplc_trip_search_home_attractions_0"]/div[2]/div[1]/div/span/input')))
    location_picker.click()
    location_picker.clear()
    location_picker.send_keys(destination)
    location_picker.send_keys(Keys.ENTER)
    
except:
    pass
#     wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="FILTERED_LIST"]/div[4]')))

# # Step 2: date picker body
# date_picker(wait, driver, dep_date, ret_date)
# wait.until(EC.element_to_be_clickable((By.ID, 'SUBMIT_ATTRACTIONS'))).click()

# Step 3: attractions url scraper
try:
    # click 'see more'
    target = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="FILTERED_LIST"]/div[4]')))
    time.sleep(5)
    ActionChains(driver).move_to_element(target).perform()
    ActionChains(driver).move_to_element(target).click().perform()
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    # get total num of pages
    page_links = soup.find_all('div',{'class', 'attractions-attraction-overview-main-Pagination__link--2m5mV attractions-attraction-overview-main-Pagination__cx_brand_refresh_phase2--3XKui'})
    page_total = int(page_links[-1].text)
    print(page_total)
    page_total = min(10, page_total)
    
except:
    driver.refresh()
    target = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="FILTERED_LIST"]/div[4]')))
    time.sleep(5)
    ActionChains(driver).move_to_element(target).perform()
    ActionChains(driver).move_to_element(target).click().perform()
    soup = BeautifulSoup(driver.page_source, 'lxml')
    # get total num of pages
    page_links = soup.find_all('div',{'class', 'attractions-attraction-overview-main-Pagination__link--2m5mV attractions-attraction-overview-main-Pagination__cx_brand_refresh_phase2--3XKui'})
    page_total = int(page_links[-1].text)
    print(page_total)
    page_total = min(10, page_total)
    
    


# initialization
fieldnames = [
    'Name', 'Url', 'Rate', 'Review'
]
df_attraction_url = pd.DataFrame(columns = fieldnames)
i = 0

for p in range(page_total):
    print('the number of page = {0}/{1}'.format(p + 1, page_total))
    soup = BeautifulSoup(driver.page_source, 'lxml')
    # only first page
    if p == 0:
        attrs = soup.find_all('li', 
                                   {'class', 'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        for attr in attrs:
            attr_name = attr.find('a', {'class', 'attractions-attraction-overview-pois-PoiInfo__name--SJ0a4'}).text[3:]
            attr_url = domain + attr.find('a', {'class', 'attractions-attraction-overview-pois-PoiInfo__name--SJ0a4'}).get('href')
            try:
                attr_rate = re.findall('[0-9]+', attr.find('div', {'class' : 'ui_poi_review_rating'}).find('span').get('class')[1])[0]
            except:
                attr_rate = 'N/A'
            try:
                attr_review_count = attr.find('span', {'class', "reviewCount styleguide-bubble-rating-BubbleRatingWithReviewCount__reviewCount--37tMc"}).text
            except:
                attr_review_count = 'N/A'
            
            df_attraction_url.loc[i] = [attr_name, attr_url, attr_rate, attr_review_count]
            i += 1
            print(i)
        
        next_page = domain + soup.find('div', 
                                       {'class', 'attractions-attraction-overview-main-Pagination__link--2m5mV attractions-attraction-overview-main-Pagination__cx_brand_refresh_phase2--3XKui ui_button primary attractions-attraction-overview-main-Pagination__button--1up7M'}).find('a').get('href')
        driver.get(next_page)
    
    elif p < (page_total - 1):
        attrs = soup.find_all('div', 
                               {'class', 'attraction_element_tall'})
        for attr in attrs:
            attr_name = attr.find('div', {'class', 'tracking_attraction_title listing_title'}).find('a').text
            attr_url = domain + attr.find('div', {'class', 'tracking_attraction_title listing_title'}).find('a').get('href')
            try:
                attr_rate = re.findall('[0-9]+', attr.find('div', {'class' : 'rs rating'}).find('span').get('class')[1])[0]
            except:
                attr_rate = 'N/A'
                
            try:
                attr_review_count = attr.find('span', {'class', "more"}).find('a').text.strip()
            except:
                attr_review_count = 'N/A'
                
            df_attraction_url.loc[i] = [attr_name, attr_url, attr_rate, attr_review_count]
            i += 1
            print(i)
        next_page = domain + soup.find('a', {'class', 'nav next rndBtn ui_button primary taLnk'}).get('href')
        driver.get(next_page)
        
    else:
        attrs = soup.find_all('li', 
                                   {'class', 'attraction_element_tall'})
        for attr in attrs:
            attr_name = attr.find('div', {'class', 'tracking_attraction_title listing_title'}).find('a').text
            attr_url = domain + attr.find('div', {'class', 'tracking_attraction_title listing_title'}).find('a').get('href')
            try:
                attr_rate = re.findall('[0-9]+', attr.find('div', {'class' : 'rs rating'}).find('span').get('class')[1])[0]
            except:
                attr_rate = 'N/A'
            
            try:
                attr_review_count = attr.find('span', {'class', "more"}).find('a').text.strip()
            except:
                attr_review_count = 'N/A'
                
            df_attraction_url.loc[i] = [attr_name, attr_url, attr_rate, attr_review_count]
            i += 1
            print(i)
        
        


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59298): Max retries exceeded with url: /session/4f7631ab19eacebb1088847a614f51c2/refresh (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x12eb69358>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [105]:
df_attraction_url

,Name,Url,Rate,Review
0,The Botanical Gardens,https://www.tripadvisor.com/Attraction_Review-...,45,"1,763 reviews"
1,Sheffield Winter Garden,https://www.tripadvisor.com/Attraction_Review-...,45,"1,911 reviews"
2,Graves Park,https://www.tripadvisor.com/Attraction_Review-...,45,973 reviews
3,"Tropical Butterfly House, Wildlife & Falconry ...",https://www.tripadvisor.com/Attraction_Review-...,45,"2,338 reviews"
4,Rivelin Valley Nature Trail,https://www.tripadvisor.com/Attraction_Review-...,45,339 reviews
5,Holly Hagg Community farm,https://www.tripadvisor.com/Attraction_Review-...,50,352 reviews
6,Crucible Theatre,https://www.tripadvisor.com/Attraction_Review-...,45,719 reviews
7,Endcliffe Park,https://www.tripadvisor.com/Attraction_Review-...,45,407 reviews
8,Kelham Island Museum,https://www.tripadvisor.com/Attraction_Review-...,45,661 reviews
9,Meadowhall Shopping Centre,https://www.tripadvisor.com/Attraction_Review-...,40,"2,021 reviews"


Blocked from the initial page, so temperaily i use the page which have input the destination

In [203]:
# attractions url scraper
domain = 'https://www.tripadvisor.com'
target_url = 'https://www.tripadvisor.co.uk/Attractions-g186338-Activities-London_England.html'
opt = webdriver.ChromeOptions()
# opt.set_headless()
opt.add_experimental_option('excludeSwitches', ['enable-automation']) # Turn on developer mode
driver = webdriver.Chrome('/Users/haorongzhu/Downloads/chromedriver', options = opt)

wait = WebDriverWait(driver, 40)
driver.get(target_url)

try:
    # click 'see more'
    target = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="FILTERED_LIST"]/div[4]')))
    time.sleep(5)
    ActionChains(driver).move_to_element(target).perform()
    ActionChains(driver).move_to_element(target).click().perform()
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    page_links = soup.find_all('div',{'class', 'attractions-attraction-overview-main-Pagination__link--2m5mV attractions-attraction-overview-main-Pagination__cx_brand_refresh_phase2--3XKui'})
    page_total = int(page_links[-1].text)
    print(page_total)
    page_total = min(10, page_total)
    
except:
    driver.refresh()
    target = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="FILTERED_LIST"]/div[4]')))
    time.sleep(5)
    ActionChains(driver).move_to_element(target).perform()
    ActionChains(driver).move_to_element(target).click().perform()
    soup = BeautifulSoup(driver.page_source, 'lxml')
    page_links = soup.find_all('div',{'class', 'attractions-attraction-overview-main-Pagination__link--2m5mV attractions-attraction-overview-main-Pagination__cx_brand_refresh_phase2--3XKui'})
    page_total = int(page_links[-1].text)
    print(page_total)
    page_total = min(10, page_total)
    
    
# get total num of pages

# initialization
fieldnames = [
    'Name', 'Url', 'Rate', 'Review', 'Price'
]
df_attraction_url = pd.DataFrame(columns = fieldnames)
i = 0

for p in range(page_total):
    print('the number of page = {0}/{1}'.format(p + 1, page_total))
    soup = BeautifulSoup(driver.page_source, 'lxml')
    # only first page
    if p == 0:
        attrs = soup.find_all('li', 
                                   {'class', 'attractions-attraction-overview-pois-PoiCard__item--3UzYK'})
        for attr in attrs:
            attr_name = attr.find('a', {'class', 'attractions-attraction-overview-pois-PoiInfo__name--SJ0a4'}).text[3:]
            attr_url = domain + attr.find('a', {'class', 'attractions-attraction-overview-pois-PoiInfo__name--SJ0a4'}).get('href')
            try:
                attr_rate = re.findall('[0-9]+', attr.find('div', {'class' : 'ui_poi_review_rating'}).find('span').get('class')[1])[0]
            except:
                attr_rate = 'N/A'
            try:
                attr_review_count = attr.find('span', {'class', "reviewCount styleguide-bubble-rating-BubbleRatingWithReviewCount__reviewCount--37tMc"}).text
                attr_review_count = re.sub('[^0-9]', "", attr_review_count)
            except:
                attr_review_count = 'N/A'
                
            try:
                ticket_price = attr.find('span', {'class', 'attractions-attraction-overview-pois-PriceFrom__amount--rE8d5'}).text
            except:
                ticket_price = 0
            
            df_attraction_url.loc[i] = [attr_name, attr_url, attr_rate, attr_review_count, ticket_price]
            i += 1
            print(i)
        
        next_page = domain + soup.find('div', 
                                       {'class', 'attractions-attraction-overview-main-Pagination__link--2m5mV attractions-attraction-overview-main-Pagination__cx_brand_refresh_phase2--3XKui ui_button primary attractions-attraction-overview-main-Pagination__button--1up7M'}).find('a').get('href')
        driver.get(next_page)
    
    elif p < (page_total - 1):
        attrs = soup.find_all('div', 
                               {'class', 'attraction_element_tall'})
        for attr in attrs:
            attr_name = attr.find('div', {'class', 'tracking_attraction_title listing_title'}).find('a').text
            attr_url = domain + attr.find('div', {'class', 'tracking_attraction_title listing_title'}).find('a').get('href')
            try:
                attr_rate = re.findall('[0-9]+', attr.find('div', {'class' : 'rs rating'}).find('span').get('class')[1])[0]
            except:
                attr_rate = 'N/A'
                
            try:
                attr_review_count = attr.find('span', {'class', "more"}).find('a').text.strip()
                attr_review_count = re.sub('[^0-9]', "", attr_review_count)
            except:
                attr_review_count = 'N/A'
            
            try:
                ticket_price = attr.find('span', {'class', 'price_from'}).text
            except:
                ticket_price = 0
                
                
            df_attraction_url.loc[i] = [attr_name, attr_url, attr_rate, attr_review_count, ticket_price]
            i += 1
            print(i)
        next_page = domain + soup.find('a', {'class', 'nav next rndBtn ui_button primary taLnk'}).get('href')
        driver.get(next_page)
        
    elif p + 1 == page_total:
        attrs = soup.find_all('div', 
                                   {'class', 'attraction_element_tall'})
        for attr in attrs:
            attr_name = attr.find('div', {'class', 'tracking_attraction_title listing_title'}).find('a').text
            attr_url = domain + attr.find('div', {'class', 'tracking_attraction_title listing_title'}).find('a').get('href')
            try:
                attr_rate = re.findall('[0-9]+', attr.find('div', {'class' : 'rs rating'}).find('span').get('class')[1])[0]
            except:
                attr_rate = 'N/A'
            
            try:
                attr_review_count = attr.find('span', {'class', "more"}).find('a').text.strip()
                attr_review_count = re.sub('[^0-9]', "", attr_review_count)
            except:
                attr_review_count = 'N/A'
            
            try:
                ticket_price = attr.find('span', {'class', 'price_from'}).text
            except:
                ticket_price = 0
                
            df_attraction_url.loc[i] = [attr_name, attr_url, attr_rate, attr_review_count, ticket_price]
            i += 1
            print(i)
        
        


78
the number of page = 1/10
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
the number of page = 2/10
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
the number of page = 3/10
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
the number of page = 4/10
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
the number of page = 5/10
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
the number of page = 6/10
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
the number of page = 7/10
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
the number of page = 8/10
211
212
213
214
215
216
217
218
219
220
221
222
223
224
2

In [226]:
df_attraction_url

,Name,Url,Rate,Review,Price
0,Tower of London,https://www.tripadvisor.com/Attraction_Review-...,45,65262,£28.90
1,Tower Bridge,https://www.tripadvisor.com/Attraction_Review-...,45,38886,£9.80
2,Churchill War Rooms,https://www.tripadvisor.com/Attraction_Review-...,45,22834,£23.00
3,National Gallery,https://www.tripadvisor.com/Attraction_Review-...,45,39646,0
4,The British Museum,https://www.tripadvisor.com/Attraction_Review-...,45,72522,0
5,Westminster Abbey,https://www.tripadvisor.com/Attraction_Review-...,45,24093,£22.00
6,V&A - Victoria and Albert Museum,https://www.tripadvisor.com/Attraction_Review-...,45,31623,0
7,St. Paul's Cathedral,https://www.tripadvisor.com/Attraction_Review-...,45,16978,£17.00
8,Natural History Museum,https://www.tripadvisor.com/Attraction_Review-...,45,36750,0
9,Borough Market,https://www.tripadvisor.com/Attraction_Review-...,45,13110,0


In [227]:
df_attraction_url.to_csv('attaction_url_London.csv')

In [130]:
df_attraction_url['Url'][0]

'https://www.tripadvisor.com/Attraction_Review-g186338-d187547-Reviews-Tower_of_London-London_England.html'

In [240]:
df = pd.read_csv('attaction_url_London.csv', index_col = 0)
df = df[df.Review > 10]
df = df[df.Rate != 'N/A']
df

/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Name,Url,Rate,Review,Price
0,Tower of London,https://www.tripadvisor.com/Attraction_Review-...,45,65262,£28.90
1,Tower Bridge,https://www.tripadvisor.com/Attraction_Review-...,45,38886,£9.80
2,Churchill War Rooms,https://www.tripadvisor.com/Attraction_Review-...,45,22834,£23.00
3,National Gallery,https://www.tripadvisor.com/Attraction_Review-...,45,39646,0
4,The British Museum,https://www.tripadvisor.com/Attraction_Review-...,45,72522,0
5,Westminster Abbey,https://www.tripadvisor.com/Attraction_Review-...,45,24093,£22.00
6,V&A - Victoria and Albert Museum,https://www.tripadvisor.com/Attraction_Review-...,45,31623,0
7,St. Paul's Cathedral,https://www.tripadvisor.com/Attraction_Review-...,45,16978,£17.00
8,Natural History Museum,https://www.tripadvisor.com/Attraction_Review-...,45,36750,0
9,Borough Market,https://www.tripadvisor.com/Attraction_Review-...,45,13110,0


In [257]:
# reading datafile
df = pd.read_csv('attaction_url_London.csv', index_col = 0)
df = df[df.Review > 10]
df = df[df.Rate != 'N/A']
total_attraction = len(df)

# initialization
fieldnames = [
                'name', 'address', 'rate', 'price','rank',
                'suggested duration', 'description', 'attraction type',
                'n_Excellent', 'n_VeryGood', 'n_Average', 'n_Poor', 'n_Terrible'
             ]
df_attraction = pd.DataFrame(columns = fieldnames)

debug = False
if debug:
    limit = 300
else:
    limit = total_attraction + 1

# browser setting
opt = webdriver.ChromeOptions()
opt.set_headless()
opt.add_experimental_option('excludeSwitches', ['enable-automation']) #开启开发者模式
driver = webdriver.Chrome('/Users/haorongzhu/Downloads/chromedriver', options=opt)
    
for idx, u in enumerate(df['Url'][:limit]):
    attraction_name = df['Name'][idx]
    attraction_rate = '.'.join(str(df['Rate'][idx]))
    attraction_price = df['Price'][idx]
    print('process = {}/{}'.format(idx + 42, total_attraction))
    driver.get(u)
#     wait = WebDriverWait(driver, 20)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # --------------------------------
    # infomation
    try:
        attraction_address = soup.find('span', 
                            {'class', 'attractions-attraction-review-atf-overview-card-Address__address--1QyPi'}).text
    except:
        attraction_address = 'N/A'
    
    try:
        attraction_rank = int(re.findall('#([0-9]+)', soup.find('div',
                                      {'class', 'attractions-attraction-review-header-attraction-review-header__popIndex--H1_nL'}).text)[0])
    except:
        attraction_rank = 'N/A'
        
    try:
        suggested_soup= soup.find_all('div', {'class', 'attractions-attraction-detail-about-card-AboutSection__sectionWrapper--3PMQg'})[1].text
        suggested_duration = int(re.findall('-([0-9]+)', suggested_soup)[0])
    except:
        suggested_duration = 'N/A'
    
    try: 
        attraction_desc = soup.find_all('div',
                             {'class' : 'attractions-attraction-review-atf-overview-card-AttractionReviewATFOverviewCard__section--2uMTX'})[1].text[:-4]
    except:
        attraction_desc = 'N/A'
    
    # --------------------------------
    # attraction_label
    try:
        attraction_label = []
        labels_soup = soup.find('div', {'class', 'attractions-attraction-review-header-AttractionLinks__detail--2-xvX'}).find_all('span')
        for l in labels_soup:
            tmp = l.text.replace(',','')
            attraction_label.append(tmp.strip())
    except:
        attraction_label = []
    
    
    # --------------------------------
    # reviews score
    try:
        reviews_count = []
        reviews_soup = soup.find_all('span',{'class' : 'location-review-review-list-parts-ReviewRatingFilter__row_num--3cSP7'})
        for r in reviews_soup:
            reviews_count.append(r.text.replace(',',''))
        
    except:
        reviews_count = []
        for i in range(5):
            reviews_count.append('N/A')
    
    
    df_attraction.loc[idx] = [attraction_name, attraction_address, attraction_rate, attraction_price, attraction_rank,
                           suggested_duration, attraction_desc, attraction_label, 
                           reviews_count[0], reviews_count[1], reviews_count[2], reviews_count[3], reviews_count[4]]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: use setter for headless property instead of set_headless


process = 42/300
process = 43/300
process = 44/300
process = 45/300
process = 46/300
process = 47/300
process = 48/300
process = 49/300
process = 50/300
process = 51/300
process = 52/300
process = 53/300
process = 54/300
process = 55/300
process = 56/300
process = 57/300
process = 58/300
process = 59/300
process = 60/300
process = 61/300
process = 62/300
process = 63/300
process = 64/300
process = 65/300
process = 66/300
process = 67/300
process = 68/300
process = 69/300
process = 70/300
process = 71/300
process = 72/300
process = 73/300
process = 74/300
process = 75/300
process = 76/300
process = 77/300
process = 78/300
process = 79/300
process = 80/300
process = 81/300
process = 82/300
process = 83/300
process = 84/300
process = 85/300
process = 86/300
process = 87/300
process = 88/300
process = 89/300
process = 90/300
process = 91/300
process = 92/300
process = 93/300
process = 94/300
process = 95/300
process = 96/300
process = 97/300
process = 98/300
process = 99/300
process = 100/

In [259]:
df_attraction

,name,address,rate,price,rank,suggested duration,description,attraction type,n_Excellent,n_VeryGood,n_Average,n_Poor,n_Terrible
0,Tower of London,London W2 England,4.5,£28.90,157,N/A,Trendy and fashionable neighborhood in London ...,"[Other, Sights & Landmarks, Neighbourhoods]",1465,1046,294,33,18
1,Tower Bridge,"Westminster Bridge Road Riverside Building, Co...",4.5,£9.80,20,2,"A trip on the London Eye is truly captivating,...","[Sights & Landmarks, Points of Interest & Land...",45136,26492,11078,2411,1210
2,Churchill War Rooms,"Kensington-Chelsea, London England",4.5,£23.00,67,N/A,"Situated west of Kensington Gardens, this city...","[Other, Sights & Landmarks, Nature & Parks, Ne...",1060,434,71,7,5
3,National Gallery,N/A,4.5,0,42,N/A,N/A,"[Museums, Speciality Museums]",909,310,55,15,6
4,The British Museum,N/A,4.5,0,58,N/A,N/A,[Nature & Parks],2027,937,115,11,6
5,Westminster Abbey,"411-412 Strand, London WC2R 0NS England",4.5,£22.00,28,N/A,Although there have been various earlier enter...,"[Concerts & Shows, Theatres]",2217,364,64,21,20
6,V&A - Victoria and Albert Museum,"27 Western Gateway, London E16 1FA England",4.5,0,83,N/A,Emirates Air Line crosses the River Thames bet...,"[Transportation, Tramways]",2240,1089,264,41,27
7,St. Paul's Cathedral,N/A,4.5,£17.00,158,N/A,N/A,"[Museums, Sights & Landmarks, Speciality Museu...",1637,1145,294,62,26
8,Natural History Museum,N/A,4.5,0,68,N/A,N/A,"[Nature & Parks, Other, Sights & Landmarks, Ne...",1336,625,92,6,4
9,Borough Market,"1 The Warehouse Hertsmere Road, London E14 4AL...",4.5,0,50,3,From Roman settlement to Docklands’ regenerati...,"[Museums, Speciality Museums, History Museums]",829,383,73,11,2


In [267]:
df_new.to_csv('London_attraction.csv', index=0)

In [265]:
df_new

,name,address,rate,price,rank,suggested duration,description,attraction type,n_Excellent,n_VeryGood,n_Average,n_Poor,n_Terrible
0,Tower of London,"St Katharine's & Wapping, London EC3N 4AB England",4.5,£28.90,8,3,"Discover London’s castle – a secure fortress, ...","[Sights & Landmarks, Points of Interest & Land...",43807,16487,3893,723,353
1,Tower Bridge,"Tower Bridge Road, London SE1 2UP England",4.5,£9.80,6,2,An iconic London landmark and one of Britain's...,"[Sights & Landmarks, Observation Decks & Tower...",25397,11247,1963,217,62
2,Churchill War Rooms,"King Charles Street, Clive Steps Westminster, ...",4.5,£23.00,2,2,Discover the secrets of the Second World War i...,"[Sights & Landmarks, Museums, Military Museums...",16697,4987,931,137,81
3,National Gallery,"Trafalgar Square, London WC2N 5DN England",4.5,0,1,3,The National Gallery houses the national colle...,"[Museums, Art Museums]",29259,8547,1483,236,121
4,The British Museum,"Great Russell Street, London WC1B 3DG England",4.5,0,4,3,"A museum of the world, for the world. Discover...","[Museums, Art Museums, Speciality Museums, His...",52669,15372,3417,695,369
5,Westminster Abbey,"20 Dean's Yard Broad Sanctuary, London SW1P 3P...",4.5,£22.00,11,2,Westminster Abbey was initially built by Henry...,"[Sights & Landmarks, Historic Sites, Architect...",16404,5645,1515,340,191
6,V&A - Victoria and Albert Museum,"Cromwell Road, London SW7 2RL England",4.5,0,5,N/A,The world's greatest museum of art and design....,"[Museums, Art Museums]",23267,6519,1387,297,153
7,St. Paul's Cathedral,"St Paul's Churchyard, London EC4M 8AD England",4.5,£17.00,13,2,Rebuilt by Christopher Wren after the great Fi...,"[Sights & Landmarks, Churches & Cathedrals]",11104,4269,1048,331,226
8,Natural History Museum,"Cromwell Road South Kensington, London SW7 5BD...",4.5,0,12,N/A,A center of scientific excellence in the disco...,"[Museums, History Museums, Natural History Mus...",24449,8652,2742,654,253
9,Borough Market,"8 Southwark Street, London SE1 1TL England",4.5,0,10,N/A,An amazing food market dating back to the 13th...,"[Shopping, Flea & Street Markets]",9068,3432,481,70,59
